In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 66 kB/s 
     |████████████████████████████████| 198 kB 50.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=39ade0b03947a086afb8a1fb3b25e4f323f9bdf055a23c219683f6b5c79f922d
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('MLLib').getOrCreate()

In [6]:
training = spark.read.csv('/content/drive/MyDrive/Data_Science/Pyspark/test1.csv', header = True, inferSchema= True)

In [7]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [8]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [10]:
# Group all the independent features to one column using Vector Assembler

from pyspark.ml.feature import VectorAssembler
featureAssembler = VectorAssembler(inputCols = ['age', 'Experience'], outputCol = 'Independent_Feature')

In [11]:
output = featureAssembler.transform(training)

In [12]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent_Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [13]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent_Feature']

In [14]:
final_data = output.select('Independent_Feature', 'Salary')

In [15]:
final_data.show()

+-------------------+------+
|Independent_Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [16]:
from pyspark.ml.regression import LinearRegression

In [17]:
train_data,test_data = final_data.randomSplit([0.75,0.25])

In [18]:
regressor = LinearRegression(featuresCol = 'Independent_Feature', labelCol = 'Salary')

In [19]:
regressor = regressor.fit(train_data)

In [20]:
regressor.coefficients

DenseVector([-102.53, 1688.6818])

In [21]:
regressor.intercept

16470.03994673731

In [22]:
# prediction

pred_results = regressor.evaluate(test_data)

In [25]:
pred_results.predictions.show()

+-------------------+------+-----------------+
|Independent_Feature|Salary|       prediction|
+-------------------+------+-----------------+
|         [30.0,8.0]| 25000|26903.59520639148|
+-------------------+------+-----------------+



In [26]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(1903.595206391481, 3623674.709796625)